# Save image for cyc

In [30]:
dataset = "../data/bulkiceHup"
filelist = "train.filelist"
model_path = "/home/supercgor/gitfile/ARAI/model/pretrain/cyclenet_v0/G_S2T_CP10_LOSS0.1023.pkl"

In [31]:
from model import build_cyc_model
from model.utils import load_model
from datasets import AFMDataset

data = AFMDataset(dataset, file_list=filelist, label = False, transform = None, img_use = None)
cyc = build_cyc_model()
load_model(cyc, model_path, True)
cyc.cuda().eval().requires_grad_(False)

print("okay")

okay


In [32]:
import torch
import os
from torch import nn
from torchvision.utils import save_image
for i in range(10):
    imgs, filename = data[i]
    imgs.unsqueeze_(0)
    translate = cyc(imgs.cuda()).sigmoid()
    for j in range(imgs.shape[2]):
        dir_path = f"{dataset}/result/{model_path.split('/')[-2]}"
        os.makedirs(dir_path, exist_ok=True)
        os.makedirs(f"{dir_path}/{filename}", exist_ok=True)
        save_image(imgs[:,:,j], f"{dir_path}/{filename}/{j}_ori.png")
        save_image(translate[:,:,j], f"{dir_path}/{filename}/{j}.png")